## 1. Environment

In [9]:
import numpy as np

from copy import copy

## 2. Class Define

In [10]:
class Env:
    def __init__(self):
        self._states = set()
        self._state = None
        self._actions = []
        self._gamma = None
        
    @property
    def states(self):
        return self._states
    
    @property
    def state_space(self):
        return self._state_shape
    
    @property
    def actions(self):
        return self._actions
    
    @property
    def action_space(self):
        return len(self._actions)
    
    @property
    def gamma(self):
        return self._gamma
    
    def _world_init(self):
        raise NotImplementedError
        
    def reset(self):
        raise NotImplementedError
    
    def step(self, state, action):
        """Return distribution and next states"""
        raise NotImplementedError
        
    def set_state(self, state):
        self._state = state


class MatrixEnv(Env):
    def __init__(self, height=4, width=4):
        super().__init__()
        
        self._action_space = 4
        self._actions = list(range(4))
        
        self._state_shape = (2,)
        self._state_shape = (height, width)
        self._states = [(i, j) for i in range(height) for j in range(width)]
        
        self._gamma = 0.9
        self._height = height
        self._width = width

        self._world_init()
        
    @property
    def state(self):
        return self._state
    
    @property
    def gamma(self):
        return self._gamma
    
    def set_gamma(self, value):
        self._gamma = value
        
    def reset(self):
        self._state = self._start_point
        
    def _world_init(self):
        # start_point
        self._start_point = (0, 0)
        self._end_point = (self._height - 1, self._width - 1)
        
    def _state_switch(self, act):
        # 0: h - 1, 1: w + 1, 2: h + 1, 3: w - 1
        if act == 0:  # up
            self._state = (max(0, self._state[0] - 1), self._state[1])
        elif act == 1:  # right
            self._state = (self._state[0], min(self._width - 1, self._state[1] + 1))
        elif act == 2:  # down
            self._state = (min(self._height - 1, self._state[0] + 1), self._state[1])
        elif act == 3:  # left
            self._state = (self._state[0], max(0, self._state[1] - 1))

    def step(self, act):
        assert 0 <= act <= 3
        
        done = False
        reward = 0.

        self._state_switch(act)
        
        if self._state == self._end_point:
            reward = 1.
            done = True

        return None, done, [reward], [self._state]

In [38]:
class ValueTable:
    def __init__(self, env):
        self._shape = env.state_space
        self._values = np.zeros(self._shape)
        self._lazy_values = np.zeros(self._shape)
        
    def lazy_update(self, s, value):
        self._lazy_values[s] = value
    
    def finish_lazy(self):
        print(self._lazy_values)
        self._values = self._lazy_values
        self._lazy_values = np.zeros(self._shape)
        
    def update(self, s, value):
        self._values[s] = value
        
    def get(self, state):
        if type(state) == list:
            # loop get
            res = [self._values[s] for s in state]
            return res
        elif type(state) == tuple:
            # return directly
            return self._values[state]

In [12]:
from collections import namedtuple


Pi = namedtuple('Pi', 'act, prob')


class Policies:
    def __init__(self, env: Env):
        self._actions = env.actions
        self._default_policy = [1 / env.action_space] * env.action_space
        self._policies = dict.fromkeys(env.states, Pi(self._actions, self._default_policy))
    
    def sample(self, state):
        if self._policies.get(state, None) is None:
            self._policies[state] = Pi(self._actions, self._default_policy)

        policy = self._policies[state]
        return np.random.choice(policy.act, p=policy.prob)
    
    def retrieve(self, state):
        return self._policies[state].prob
    
    def update(self, state, policy):
        self._policies[state] = self._policies[state]._replace(prob=policy)

## 3. Iteration Functions Define

In [43]:
def policy_eval(env: MatrixEnv, values: ValueTable, policies: Policies, upper_bound: float) -> None:
    
    ## counter
    iter = 0
    
    delta = 0
    
    while delta < upper_bound:
        
        ## set delta
        delta = 0
    
        for s in env.states:
            ## Get v    
            v = values.get(s)

            ## Get V(s)
            env.set_state(s)
            probs = policies.retrieve(s)

            V_s = 0
            for act in env.actions:
                env.set_state(s)
                _, _, r, states = env.step(act)
                # print(f'im strange {states} and reward is {r}')
                v_prime = values.get(env.state)
                # print(f'im now at {env.state}')

                ## Note here my leads to looop

                ## Bellman Equation
                V_s += probs[act] * (r + env.gamma * v_prime)
                
            values.lazy_update(s, V_s)

            delta = max(delta, abs(v - V_s))
            
        # print(delta)
            
        values.finish_lazy()
        
        iter += 1
        
        print('\r> iteration: {} delta: {}'.format(iter, delta), flush=True, end="")


In [25]:
def policy_improve(env: MatrixEnv, values: ValueTable, policies: Policies) -> bool:
    
    stable = True
    
    for s in env.states:
        
        probs = policies.retrieve(s)
        
        sums = []
        
        for act in env.actions:
            
            env.set_state(s)
            _, _, r, states = env.step(act)
            
            v_prime = values.get(env.state)
            
            sums.append(probs[act] * (r + env.gamma * v_prime))
            
        better_act_tmp = env.actions[np.argmax(sums)]
        
        equal_better_acts = []
        
        for i in range(env.action_space):
            if sums[i] == sums[better_act_tmp]:
                equal_better_acts.append(i)
        
        new_policy = [0] * env.action_space
        
        for act in equal_better_acts:
            new_policy[act] = 1/len(equal_better_acts)
        
        policies.update(s, new_policy)
        
        for i in range(len(probs)):
            if probs[i] != new_policy[i]:
                stable = False
                
            # print(probs, '~~~', new_policy)
            
        ## 需要单步调试 看看哪里出问题了
        
                
    return stable
    
    # pi(s) is an approximate action

In [15]:
# probs = policies.retrieve((0, 0))
# print(probs)

# sums = []
        
# for act in env.actions:
    
#     env.set_state(s)
#     _, _, r, states = env.step(act)
    
#     v_prime = values.get(env.state)
    
#     sums.append(probs[act] * (r + env.gamma * v_prime))
    

    

In [16]:
def value_iter(env: MatrixEnv, values: ValueTable, upper_bound: Policies) -> None:
    pass


In [46]:
import time

env = MatrixEnv(width=8, height=8)  # TODO(ming): try different word size
policies = Policies(env)
values = ValueTable(env)
upper_bound = 1e-4

stable = False

start = time.time()

iter = 0
while not stable:
    policy_eval(env, values, policies, upper_bound)
    # print('imhere')
    stable = policy_improve(env, values, policies)
    iter += 1
    # print(iter)
    

end = time.time()

print('\n[time consumpution]: {} s'.format(end - start))

done = False
rewards = 0
env.reset()
step = 0

while not done:
    act_index = policies.sample(env.state)
    _, done, reward, next_state = env.step(env.actions[act_index])
    rewards += sum(reward)
    step += 1

print('Evaluation: [reward] {} [step] {}'.format(rewards, step))

[[0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25]
 [0.   0.   0.   0.   0.   0.   0.25 0.5 ]]
> iteration: 1 delta: [0.5][[0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.225]
 [0.    0.    0.    0.    0.    0.    0.225 1.45 ]
 [0.    0.    0.    0.    0.    0.225 1.45  1.45 ]]
> iteration: 1 delta: [1.2][[0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.    

In [47]:
for s in env.states:
    # print(s, policies.retrieve(s))
    print(s, values.get(s))

(0, 0) 0.06354664570822503
(0, 1) 0.40952282789745004
(0, 2) 0.7233334239874502
(0, 3) 1.0720118640874503
(0, 4) 1.4594323530874505
(0, 5) 1.8898995630874502
(0, 6) 2.368196463087451
(0, 7) 2.8996374630874504
(1, 0) 0.40952282789745004
(1, 1) 0.7233334239874502
(1, 2) 1.0720118640874503
(1, 3) 1.4594323530874505
(1, 4) 1.8898995630874502
(1, 5) 2.368196463087451
(1, 6) 2.8996374630874504
(1, 7) 3.4901274630874504
(2, 0) 0.7233334239874502
(2, 1) 1.0720118640874503
(2, 2) 1.4594323530874505
(2, 3) 1.8898995630874502
(2, 4) 2.368196463087451
(2, 5) 2.8996374630874504
(2, 6) 3.4901274630874504
(2, 7) 4.146227463087451
(3, 0) 1.0720118640874503
(3, 1) 1.4594323530874505
(3, 2) 1.8898995630874502
(3, 3) 2.368196463087451
(3, 4) 2.8996374630874504
(3, 5) 3.4901274630874504
(3, 6) 4.146227463087451
(3, 7) 4.875227463087451
(4, 0) 1.4594323530874505
(4, 1) 1.8898995630874502
(4, 2) 2.368196463087451
(4, 3) 2.8996374630874504
(4, 4) 3.4901274630874504
(4, 5) 4.146227463087451
(4, 6) 4.875227463

In [19]:
print(values._values)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [20]:
s = (0,0)

v = values.get(s)
        
## Get V(s)
env.set_state(s)
probs = policies.retrieve(s)

V_s = 0
for act in env.actions:
    env.set_state(s)
    _, _, r, states = env.step(act)
    print(f'im strange {states} and reward is {r}')
    v_prime = values.get(env.state)
    print(f'im now at {env.state}')
    
    ## Note here my leads to looop
    
    V_s += probs[act] * (r + env.gamma * v_prime)
    
print(V_s)
        

im strange [(0, 0)] and reward is [0.0]
im now at (0, 0)
im strange [(0, 1)] and reward is [0.0]
im now at (0, 1)
im strange [(1, 0)] and reward is [0.0]
im now at (1, 0)
im strange [(0, 0)] and reward is [0.0]
im now at (0, 0)
[0.]


In [21]:
env = MatrixEnv(width=8, height=8)  # try different word size
policies = Policies(env)
values = ValueTable(env)
upper_bound = 1e-4

start = time.time()
value_iter(env, values, upper_bound)
_ = policy_improve(env, values, policies)
end = time.time()

print('\n[time consumption] {}s'.format(end - start))
# print("===== Render =====")
env.reset()
done = False
rewards = 0
step = 0
while not done:
    act_index = policies.sample(env.state)
    _, done, reward, next_state = env.step(env.actions[act_index])
    rewards += sum(reward)
    step += 1

print('Evaluation: [reward] {} [step] {}'.format(rewards, step))


[time consumption] 0.011972427368164062s
Evaluation: [reward] 1.0 [step] 135
